In [ ]:
import numpy as np
import tensorflow as tf
import _pickle as pickle


In [ ]:
# player 0, player 1
player1 = np.array([[[1, 0, 0], [1, 0, 1], [1, 1, 0]]])
player2 = np.array([[[0, 1, 1], [0, 1, 0], [0, 0, 1]]])
# multi class label
# [1, 0] player 1 wins
# [0, 1] player 2 wins
# [0, 0] draw
outcome = np.array([[1, 0]])

In [ ]:
player2

In [ ]:
# tf estimator
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"player1": player1, "player2": player2},
    y=outcome,
    num_epochs=1,
    shuffle=True) 


In [ ]:
train_input_fn()

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
x, y = train_input_fn()
tf.global_variables_initializer()

In [ ]:
#x_vals = sess.run(x)

In [ ]:
array_player_1 = np.full((1000, 3, 3), -2)
array_player_2 = np.full((1000, 3, 3), -2)

In [ ]:
for i in range(1000):
    array_player_1[i] = [[1, 0, 0], [1, 0, 1], [1, 1, 0]]
    array_player_2[i] = [[0, 1, 1], [0, 1, 0], [0, 0, 1]]


In [ ]:
the_data = {'player1': array_player_1, 'player2': array_player_2, 'y': outcome}
with open(filein, 'wb') as f:
    preview = pickle.dump(the_data, f)